Mini Project - Modeling (2022-06-07 ~ 06-21)

# IEEE-CIS Fraud Detection
Can you detect fraud from customer transactions?

https://www.kaggle.com/competitions/ieee-fraud-detection/data



[Data Description (Details and Discussion)](https://www.kaggle.com/competitions/ieee-fraud-detection/discussion/101203)

## **Transaction Table**
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.


<br>

---

### *Numerical Features:*
> - TransactionID :  거래 ID
> - isFraud :  사기 여부 (정상 0, 사기 1)
> - TransactionDT :  주어진 참조 datetime의 timedelta(실제 타임스탬프가 아님) 초단위
> (TransactionDT의 첫 번째 값은 86400이며, 이는 하루의 초 수(60 60 24 = 86400)에 해당합니다. 따라서 최대 값이 15811131이고 183일째에 해당하므로 데이터가 6개월에 걸쳐 있음을 알고 있습니다.")
> - TransactionAmt :  USD 거래 결제 금액
> - dist1, dist2 :  거리 (청구주소, 우편주소, 우편번호, IP주소, 전화 지역간의 거리...)
> - card1,2,3,5 : 카드종류(type), 카드분류(category), 발행 은행, 국가 등 결제 지불 카드에 대한 정보
> - C1 - C14 : (Counting) 계산, 예를 들어 결제 카드와 연결된 주소가 몇 개인지 등 (실제 의미는 가려짐)
> - D1 - D15 : (Timedelta) 이전 거래 사이의 일수 등과 같은 timedelta
> - V1 - V339 :  Vesta가 설계한 순위, 계산 및 기타 엔터티 관계를 포함하여 여러 기능

### *Categorical Features:*
> - ProductCD (5) : 제품 코드, 각 거래에 대한 제품
> - **card4 (4)**: 글로벌 결제서비스 카드사 (discover/mastercard/visa/american express/nan)
>  - **card6 (4)**: 카드결제방식에 따른 분류 (신용카드/직불카드 등 : credit/debit/debit or credit/charge card/nan)
>> - [결제방식에 따라 이용하는 전산망 다름](https://namu.wiki/w/%EC%A7%81%EB%B6%88%EC%B9%B4%EB%93%9C)
>> - [Payment card number](https://en.wikipedia.org/wiki/Payment_card_number)  
>>- [카드 번호의 구성](https://ko.wikipedia.org/wiki/%EC%B9%B4%EB%93%9C_%EB%B2%88%ED%98%B8%EC%9D%98_%EA%B5%AC%EC%84%B1)
> - addr1, addr2 : (구매자에 대한 정보) 청구서 발행지 지역, 청구서 발행 국가 | addr1 국가, addr2 우편번호
> - P_emaildomain (59): 구매자 이메일 도메인  **[변수 줄이기]**
> - R_emaildomain (60): 수신자 이메일 도메인  **[변수 줄이기]**
> - M1 - M9 (2-3): (Match) 카드의 이름 및 주소 등과 같은 일치



## **Identity Table**  

Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
They're collected by Vesta’s fraud protection system and digital security partners.
(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

이 표의 변수는 ID 정보 - 트랜잭션과 관련된 네트워크 연결 정보(IP, ISP, 프록시 등) 및 디지털 서명(UA/브라우저/OS/버전 등)입니다.
Vesta의 사기 방지 시스템과 디지털 보안 파트너가 수집합니다.
(필드 이름은 마스킹되며 개인 정보 보호 및 계약 합의를 위해 쌍별 사전은 제공되지 않습니다.)




### *Numerical Features:*
> - TransactionID :  거래 ID
> - id_01 - id_11
> - id_13 - id_14
> - id_17 - id_22
> - id_24 - id_26
> - id_32 - id_37


### *Categorical Features:*
> - id_12 (2)  
> - id_15 (3)-16 (2)
> - id_23 (3)
> - id_27-29 (2)
> - id_30 (75)  **[변수 줄이기]**
> - id_31 (130)  **[변수 줄이기]**
> - id_33 (260)  **[변수 줄이기]**
> - id_34 (4)
> - id_35-38 (2) True:1, False:0 으로 대입하기  (Nan: ???)
> - DeviceType (2): 거래 기기 유형 (mobile/desktop/NaN)
> - DeviceInfo (1786) : 거래 기기 정보  **[변수 줄이기]**


## 1. 라이브러리, 데이터 불러오기

In [156]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [157]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
from sklearn.ensemble import RandomForestRegressor
#from lightgbm.sklearn import LGBMRegressor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

import warnings
warnings.filterwarnings('ignore')

In [158]:
# flag setting
feature_reducing = "feature_importance" # "correlation" / "feature_importance" / "PCA"

In [159]:
# 데이터를 불러옵니다.
base_path = "./ieee-fraud-detection/"

train_identity = pd.read_csv(base_path + "train_identity.csv")
train_transaction = pd.read_csv(base_path + "train_transaction.csv")

test_identity = pd.read_csv(base_path + "test_identity.csv")
test_transaction = pd.read_csv(base_path + "test_transaction.csv")

In [160]:
print(train_identity.shape, test_identity.shape, train_transaction.shape, test_transaction.shape)

(144233, 41) (141907, 41) (590540, 394) (506691, 393)


### Merge : identity + transaction

In [161]:
df_train = pd.merge(train_identity, train_transaction, how='right', on='TransactionID')
df_train.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. EDA

1. 결측치 : 엄청 많음  
  - 결측치 없는 column : 3개 (TransactionID, id_01, id_12)
  - 결측치 없는 column: 20개 /394개 중

2. dtype이 object인 column (categorical feature)  
→ 어떻게 처리할지 고민해야함. (Ordinal Encoding VS One-Hot Encoding)

3. Target값 (isFraud == 1)과 관련 있어보이는 컬럼 찾기

4. target distribution (Outlier 찾기)

In [162]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 434 entries, TransactionID to V339
dtypes: float64(399), int64(4), object(31)
memory usage: 1.9+ GB


In [163]:
df_train.describe(include=['O'])

,id_12,id_15,id_16,id_23,id_27,id_28,id_29,id_30,id_31,id_33,...,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
count,144233,140985,129340,5169,5169,140978,140978,77565,140282,73289,...,137291,319440,319440,319440,309096,240058,421180,244275,244288,244288
unique,2,3,2,3,2,2,2,75,130,260,...,60,2,2,2,3,2,2,2,2,2
top,NotFound,Found,Found,IP_PROXY:TRANSPARENT,Found,Found,Found,Windows 10,chrome 63.0,1920x1080,...,gmail.com,T,T,T,M0,F,F,F,F,T
freq,123025,67728,66324,3489,5155,76232,74926,21155,22000,16874,...,57147,319415,285468,251731,196405,132491,227856,211374,155251,205656


In [164]:
df_train.describe()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,144233.000000,140872.000000,66324.000000,66324.000000,136865.000000,136865.000000,5155.000000,5155.000000,74926.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,3.282270e+06,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.698710,13.285354,-38.600388,0.091023,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,2.987000e+06,-100.000000,1.000000,-13.000000,-28.000000,-72.000000,-100.000000,-46.000000,-100.000000,-36.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,-10.000000,67992.000000,0.000000,0.000000,0.000000,-6.000000,5.000000,-48.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,3.282270e+06,-5.000000,125800.500000,0.000000,0.000000,0.000000,0.000000,14.000000,-34.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,3.429904e+06,-5.000000,228749.000000,0.000000,0.000000,1.000000,0.000000,22.000000,-23.000000,0.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,0.000000,999595.000000,10.000000,0.000000,52.000000,0.000000,61.000000,0.000000,25.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


### Identity

In [165]:
train_identity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  object 
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  object 
 16  id_16          129340 non-null  object 
 17  id_17          139369 non-nul

In [166]:
train_identity.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object')

In [167]:
train_identity.isna().sum()

# 결측치 없는 column : TransactionID, id_01, id_12

TransactionID         0
id_01                 0
id_02              3361
id_03             77909
id_04             77909
id_05              7368
id_06              7368
id_07            139078
id_08            139078
id_09             69307
id_10             69307
id_11              3255
id_12                 0
id_13             16913
id_14             64189
id_15              3248
id_16             14893
id_17              4864
id_18             99120
id_19              4915
id_20              4972
id_21            139074
id_22            139064
id_23            139064
id_24            139486
id_25            139101
id_26            139070
id_27            139064
id_28              3255
id_29              3255
id_30             66668
id_31              3951
id_32             66647
id_33             70944
id_34             66428
id_35              3248
id_36              3248
id_37              3248
id_38              3248
DeviceType         3423
DeviceInfo        25567
dtype: int64

In [168]:
train_identity.isna().any().sum()

38

In [169]:
train_identity.describe()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,1.442330e+05,144233.000000,140872.000000,66324.000000,66324.000000,136865.000000,136865.000000,5155.000000,5155.000000,74926.000000,...,139369.000000,45113.000000,139318.000000,139261.000000,5159.000000,5169.000000,4747.000000,5132.000000,5163.000000,77586.000000
mean,3.236329e+06,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.698710,13.285354,-38.600388,0.091023,...,189.451377,14.237337,353.128174,403.882666,368.269820,16.002708,12.800927,329.608924,149.070308,26.508597
std,1.788496e+05,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,...,30.375360,1.561302,141.095343,152.160327,198.847038,6.897665,2.372447,97.461089,32.101995,3.737502
min,2.987004e+06,-100.000000,1.000000,-13.000000,-28.000000,-72.000000,-100.000000,-46.000000,-100.000000,-36.000000,...,100.000000,10.000000,100.000000,100.000000,100.000000,10.000000,11.000000,100.000000,100.000000,0.000000
25%,3.077142e+06,-10.000000,67992.000000,0.000000,0.000000,0.000000,-6.000000,5.000000,-48.000000,0.000000,...,166.000000,13.000000,266.000000,256.000000,252.000000,14.000000,11.000000,321.000000,119.000000,24.000000
50%,3.198818e+06,-5.000000,125800.500000,0.000000,0.000000,0.000000,0.000000,14.000000,-34.000000,0.000000,...,166.000000,15.000000,341.000000,472.000000,252.000000,14.000000,11.000000,321.000000,149.000000,24.000000
75%,3.392923e+06,-5.000000,228749.000000,0.000000,0.000000,1.000000,0.000000,22.000000,-23.000000,0.000000,...,225.000000,15.000000,427.000000,533.000000,486.500000,14.000000,15.000000,371.000000,169.000000,32.000000
max,3.577534e+06,0.000000,999595.000000,10.000000,0.000000,52.000000,0.000000,61.000000,0.000000,25.000000,...,229.000000,29.000000,671.000000,661.000000,854.000000,44.000000,26.000000,548.000000,216.000000,32.000000


In [170]:
train_identity.describe(include=['O'])

,id_12,id_15,id_16,id_23,id_27,id_28,id_29,id_30,id_31,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
count,144233,140985,129340,5169,5169,140978,140978,77565,140282,73289,77805,140985,140985,140985,140985,140810,118666
unique,2,3,2,3,2,2,2,75,130,260,4,2,2,2,2,2,1786
top,NotFound,Found,Found,IP_PROXY:TRANSPARENT,Found,Found,Found,Windows 10,chrome 63.0,1920x1080,match_status:2,T,F,T,F,desktop,Windows
freq,123025,67728,66324,3489,5155,76232,74926,21155,22000,16874,60011,77814,134066,110452,73922,85165,47722


In [171]:
train_identity.DeviceType.unique()

array(['mobile', 'desktop', nan], dtype=object)

---

### Transaction

In [172]:
train_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [173]:
train_transaction.columns.unique()

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=394)

In [174]:
train_transaction.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


In [175]:
train_transaction.describe(include=['O'])

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
count,590540,588963,588969,496084,137291,319440,319440,319440,309096,240058,421180,244275,244288,244288
unique,5,4,4,59,60,2,2,2,3,2,2,2,2,2
top,W,visa,debit,gmail.com,gmail.com,T,T,T,M0,F,F,F,F,T
freq,439670,384767,439938,228355,57147,319415,285468,251731,196405,132491,227856,211374,155251,205656


### each feature (train_transaction)

In [176]:
train_transaction.iloc[:, :11].head(30)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6
0,2987000,0,86400,68.500,W,13926,NaN,150.0,discover,142.0,credit
1,2987001,0,86401,29.000,W,2755,404.0,150.0,mastercard,102.0,credit
2,2987002,0,86469,59.000,W,4663,490.0,150.0,visa,166.0,debit
3,2987003,0,86499,50.000,W,18132,567.0,150.0,mastercard,117.0,debit
4,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,credit
5,2987005,0,86510,49.000,W,5937,555.0,150.0,visa,226.0,debit
6,2987006,0,86522,159.000,W,12308,360.0,150.0,visa,166.0,debit
7,2987007,0,86529,422.500,W,12695,490.0,150.0,visa,226.0,debit
8,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,debit
9,2987009,0,86536,117.000,W,17399,111.0,150.0,mastercard,224.0,debit


#### card1 - card6

> 참고
 - https://www.kaggle.com/code/kabure/extensive-eda-and-modeling-xgb-hyperopt#P-emaildomain-Distributions  
 - https://www.forbes.com/advisor/credit-cards/what-does-your-credit-card-number-mean/  
 - https://wppbaz.com/american-express-card-number-format/web/  
 - https://www.creditcardvalidator.org/visa-issued  


첫째 자리부터 여섯째 자리까지의 번호(BIN 번호)만 보면 해당 카드는 
어느 나라의 어느 카드사가 발급한 카드인지, 카드 회원이 일반, 골드, 개인, 법인인지 알 수 있도록 되어 있다.

[1] MII 숫자값: 주 산업 식별변호, 카드번호의 첫 번째 자리에 해당하는 숫자이다. 앞자리 4는 비자카드로만 나오며 9는 해외결제불가이다.

[2-7, 2-9] 발급자 식별 번호(IIN): 카드의 발급자를 식별할 수 있는 6자리 또는 8자리로 된 번호이다. 「은행 식별 번호(BIN)」라고도 불립니다.

가변 길이(최대 12자리)의 개별 계정 식별자

카드사 / IIN 범위 / 전체 자릿수  
- 아멕스	34, 37	/ 15  
- 비자카드	4	/ 16  
- 마스터카드	51-55(2221-2720 2017년부터-)  / 16  
- 디스커버	60110, 60112-60114, 601174-601179, 601186-601199, 644-649, 65(60,61,64,65)	/ 16  
'''

나머지 숫자:  
7번째 자리부터 15번째 자리까지는 각 카드사가 임의의 규칙에 따라 사용하도록 되어 있고 16번째 숫자는 특정한 공식에 의해 카드 번호를 검증하는 값이다.  
이 특정한 공식을 룬 공식(LUHN Formula) 이라고 한다. 그 공식은 다음과 같다.

> (홀수 자리 숫자×2) + (짝수 자리 숫자의 합)  
EX) 4520 0200 1900 4060 (홀수 자리 숫자×2)+짝수 자리 숫자의 합= (4×2)+(2×2)+(0×2)+(0×2)+(1×2)+(0×2)+(4×2)+(6×2)+5+0+2+0+9+0+0=50  
50이 10으로 나누어떨어지므로 유효한 번호

카드보안코드  
카드 뒷면의 서명판에 기울여서 인쇄된 3자리(비자, 마스타)/4자리(아멕스) 숫자는 카드소지를 확인하는 값으로 카드사마다 부르는 이름이 다르다.

비자카드: CVV(Card Verification Value)
마스터카드, JCB: CVC(Card Validation Code)
아멕스: CID(Confidential Identifier Number 또는 Card Identification Number)
CVV나 CVC, CID는 모두 카드 번호가 제대로 된 것인지를 확인하는 값으로 요즈음은 인터넷 상에서 상거래 시 입력을 요구하는 경우도 있다. 이 3자리 값과 16자리 카드 번호를 정해진 규칙에 따라 암복호화를 하여 해당 값이 일치하면 카드가 정상 카드임을 알 수 있는 것이다.

CVV나 CVC, CID는 MS에 기록되는 CVV1(CVC1)과 카드의 뒷면 서명판 윗 부분에 음각되는 CVV2(CVC2)의 값이 있다. 아메리칸 엑스프레스의 경우 카드의 앞면 오른쪽 중간부분에 4자리의 CID 값이 있다. 물론 EMV표준의 IC Chip에는 iCVV(Chip CVC)의 값이 입력되어 있다.

In [177]:
# card1 -card6
train_transaction.iloc[:, 5:11].head(30)

,card1,card2,card3,card4,card5,card6
0,13926,NaN,150.0,discover,142.0,credit
1,2755,404.0,150.0,mastercard,102.0,credit
2,4663,490.0,150.0,visa,166.0,debit
3,18132,567.0,150.0,mastercard,117.0,debit
4,4497,514.0,150.0,mastercard,102.0,credit
5,5937,555.0,150.0,visa,226.0,debit
6,12308,360.0,150.0,visa,166.0,debit
7,12695,490.0,150.0,visa,226.0,debit
8,2803,100.0,150.0,visa,226.0,debit
9,17399,111.0,150.0,mastercard,224.0,debit


In [178]:
card = train_transaction.iloc[:, 5:11]
card.corr()

,card1,card2,card3,card5
card1,1.000000,0.004960,0.002965,-0.093633
card2,0.004960,1.000000,0.023816,0.030486
card3,0.002965,0.023816,1.000000,-0.158383
card5,-0.093633,0.030486,-0.158383,1.000000


In [179]:
card[card.card4 =='american express'].value_counts()

card1  card2  card3  card4             card5  card6 
5957   520.0  150.0  american express  190.0  credit    74
14608  453.0  150.0  american express  131.0  credit    73
7164   399.0  150.0  american express  150.0  credit    66
8937   399.0  150.0  american express  150.0  credit    60
7709   399.0  150.0  american express  150.0  credit    57
                                                        ..
6703   520.0  150.0  american express  190.0  credit     1
4026   399.0  150.0  american express  137.0  credit     1
4061   555.0  197.0  american express  145.0  credit     1
15089  520.0  150.0  american express  190.0  credit     1
7809   399.0  150.0  american express  132.0  credit     1
Length: 1279, dtype: int64

In [180]:
card['card1'].value_counts()

7919     14932
9500     14162
15885    10361
17188    10344
15066     7945
         ...  
17084        1
14620        1
11440        1
13231        1
18038        1
Name: card1, Length: 13553, dtype: int64

In [181]:
card['card2'].value_counts()

321.0    48935
111.0    45191
555.0    41995
490.0    38145
583.0    21803
         ...  
388.0       39
153.0       38
557.0       37
473.0       18
582.0       14
Name: card2, Length: 500, dtype: int64

In [182]:
card['card3'].value_counts()

150.0    521287
185.0     56346
106.0      1571
144.0      1252
146.0      1252
          ...  
224.0         1
199.0         1
221.0         1
209.0         1
173.0         1
Name: card3, Length: 114, dtype: int64

In [183]:
card['card4'].value_counts()

visa                384767
mastercard          189217
american express      8328
discover              6651
Name: card4, dtype: int64

In [184]:
card['card5'].value_counts()

226.0    296546
224.0     81513
166.0     57140
102.0     29105
117.0     25941
          ...  
221.0         1
167.0         1
115.0         1
165.0         1
234.0         1
Name: card5, Length: 119, dtype: int64

In [185]:
train_transaction.iloc[293132]

TransactionID     3280132
isFraud                 0
TransactionDT     7239145
TransactionAmt     44.348
ProductCD               C
                   ...   
V335                  NaN
V336                  NaN
V337                  NaN
V338                  NaN
V339                  NaN
Name: 293132, Length: 394, dtype: object

In [186]:
card['card6'].value_counts()

debit              439938
credit             148986
debit or credit        30
charge card            15
Name: card6, dtype: int64

In [187]:
print(f'card4 고유값: {card.card4.unique()}')
print()
print(f'card6 고유값: {card.card6.unique()}')

card4 고유값: ['discover' 'mastercard' 'visa' 'american express' nan]

card6 고유값: ['credit' 'debit' nan 'debit or credit' 'charge card']


In [188]:
card_copy = card.copy()

In [189]:
from sklearn.preprocessing import LabelEncoder    # LabelEncoder를 불러오기
encoder = LabelEncoder()      # encoder라는 변수를 선언
encoder.fit(card_copy['card4']) # title 열에 맞게 인코딩
card_copy['card4'] = encoder.transform(card_copy['card4']) # 인코딩 결과를 실제 행에 적용
card_copy.head(10)

,card1,card2,card3,card4,card5,card6
0,13926,NaN,150.0,1,142.0,credit
1,2755,404.0,150.0,2,102.0,credit
2,4663,490.0,150.0,3,166.0,debit
3,18132,567.0,150.0,2,117.0,debit
4,4497,514.0,150.0,2,102.0,credit
5,5937,555.0,150.0,3,226.0,debit
6,12308,360.0,150.0,3,166.0,debit
7,12695,490.0,150.0,3,226.0,debit
8,2803,100.0,150.0,3,226.0,debit
9,17399,111.0,150.0,2,224.0,debit


In [190]:
encoder.inverse_transform([0, 1, 2, 3, 4])

array(['american express', 'discover', 'mastercard', 'visa', nan],
      dtype=object)

In [191]:
from sklearn.preprocessing import LabelEncoder    # LabelEncoder를 불러오기
encoder = LabelEncoder()      # encoder라는 변수를 선언
encoder.fit(card_copy['card6']) # title 열에 맞게 인코딩
card_copy['card6'] = encoder.transform(card_copy['card6']) # 인코딩 결과를 실제 행에 적용
card_copy.head(10)

,card1,card2,card3,card4,card5,card6
0,13926,NaN,150.0,1,142.0,1
1,2755,404.0,150.0,2,102.0,1
2,4663,490.0,150.0,3,166.0,2
3,18132,567.0,150.0,2,117.0,2
4,4497,514.0,150.0,2,102.0,1
5,5937,555.0,150.0,3,226.0,2
6,12308,360.0,150.0,3,166.0,2
7,12695,490.0,150.0,3,226.0,2
8,2803,100.0,150.0,3,226.0,2
9,17399,111.0,150.0,2,224.0,2


In [192]:
encoder.inverse_transform([0, 1, 2, 3, 4])

array(['charge card', 'credit', 'debit', 'debit or credit', nan],
      dtype=object)

In [193]:
ccc = card_copy.corr()
find_num = 0.2
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
ccc.style.applymap(draw_color, color='green')

,card1,card2,card3,card4,card5,card6
card1,1.000000,0.004960,0.002965,0.048783,-0.093633,0.019827
card2,0.004960,1.000000,0.023816,0.028596,0.030486,-0.134365
card3,0.002965,0.023816,1.000000,-0.012501,-0.158383,-0.067028
card4,0.048783,0.028596,-0.012501,1.000000,0.387050,0.224466
card5,-0.093633,0.030486,-0.158383,0.387050,1.000000,0.167092
card6,0.019827,-0.134365,-0.067028,0.224466,0.167092,1.000000


#### C1 - C14

In [194]:
# C1-C14
train_transaction.iloc[: , 17:31].head(20)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0
1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0
4,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
5,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
7,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
9,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0,12.0,2.0


In [195]:
C = train_transaction.iloc[:, 17:31]
C

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0
1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0
4,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,2.0
590536,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
590537,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
590538,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,1.0,1.0,5.0,1.0


In [196]:
C.value_counts()

C1      C2      C3   C4      C5   C6      C7      C8      C9   C10     C11     C12     C13     C14   
1.0     1.0     0.0  0.0     0.0  1.0     0.0     0.0     1.0  0.0     1.0     0.0     1.0     1.0       41367
                     1.0     0.0  1.0     0.0     1.0     0.0  1.0     1.0     0.0     1.0     1.0       34815
                                          1.0     1.0     0.0  1.0     1.0     1.0     1.0     1.0       19550
                     0.0     1.0  1.0     0.0     0.0     1.0  0.0     1.0     0.0     1.0     1.0       18639
                             0.0  1.0     0.0     0.0     0.0  0.0     1.0     0.0     1.0     1.0       11538
                                                                                                         ...  
4.0     2.0     0.0  0.0     0.0  0.0     0.0     0.0     1.0  0.0     2.0     0.0     5.0     3.0           1
                                                                       1.0     0.0     14.0    4.0           1
          

In [197]:
C.describe()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
count,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,14.092458,15.269734,0.005644,4.092185,5.571526,9.071082,2.848478,5.144574,4.480240,5.240343,10.241521,4.076227,32.539918,8.295215
std,133.569018,154.668899,0.150536,68.848459,25.786976,71.508467,61.727304,95.378574,16.674897,95.581443,94.336292,86.666218,129.364844,49.544262
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
50%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,1.000000
75%,3.000000,3.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,12.000000,2.000000
max,4685.000000,5691.000000,26.000000,2253.000000,349.000000,2253.000000,2255.000000,3331.000000,210.000000,3257.000000,3188.000000,3188.000000,2918.000000,1429.000000


In [198]:
C.corr()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
C1,1.000000,0.995089,-0.003467,0.967800,0.165862,0.982238,0.926258,0.967746,0.175999,0.958202,0.996515,0.927939,0.774603,0.951761
C2,0.995089,1.000000,-0.003339,0.972134,0.126540,0.974845,0.938867,0.975863,0.133566,0.970624,0.993898,0.940258,0.751221,0.936148
C3,-0.003467,-0.003339,1.000000,-0.001720,-0.008101,-0.004711,-0.001730,-0.001203,-0.010074,-0.001494,-0.003583,-0.001763,-0.007747,-0.005091
C4,0.967800,0.972134,-0.001720,1.000000,-0.012842,0.962319,0.895092,0.959995,-0.015970,0.952466,0.974547,0.894619,0.644549,0.907676
C5,0.165862,0.126540,-0.008101,-0.012842,1.000000,0.232409,-0.009970,-0.011654,0.925786,-0.011846,0.168862,-0.009875,0.717509,0.378950
C6,0.982238,0.974845,-0.004711,0.962319,0.232409,1.000000,0.858583,0.921972,0.250695,0.914440,0.991105,0.858182,0.808531,0.984201
C7,0.926258,0.938867,-0.001730,0.895092,-0.009970,0.858583,1.000000,0.982983,-0.012399,0.985062,0.915209,0.999489,0.632394,0.794701
C8,0.967746,0.975863,-0.001203,0.959995,-0.011654,0.921972,0.982983,1.000000,-0.014492,0.996970,0.962722,0.983027,0.653245,0.860246
C9,0.175999,0.133566,-0.010074,-0.015970,0.925786,0.250695,-0.012399,-0.014492,1.000000,-0.014731,0.182446,-0.012212,0.704056,0.397396
C10,0.958202,0.970624,-0.001494,0.952466,-0.011846,0.914440,0.985062,0.996970,-0.014731,1.000000,0.956056,0.983817,0.653941,0.853009


#### D1 - D15

In [199]:
# D1-D15
train_transaction.iloc[: , 31:46].head(20)

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0
1,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
2,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0
3,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0
6,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0
7,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,61.0,61.0,30.0,318.0,30.0,NaN,NaN,NaN,NaN,40.0,302.0,NaN,NaN,NaN,318.0


In [200]:
D = train_transaction.iloc[: , 31:46]
D.head()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0
1,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
2,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,315.0,NaN,NaN,NaN,315.0
3,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,111.0
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
D.describe()

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
count,589271.000000,309743.000000,327662.000000,421618.000000,280699.000000,73187.000000,38917.000000,74926.000000,74926.000000,514518.000000,311253.000000,64717.000000,61952.000000,62187.000000,501427.000000
mean,94.347568,169.563231,28.343348,140.002441,42.335965,69.805717,41.638950,146.058108,0.561057,123.982137,146.621465,54.037533,17.901295,57.724444,163.744579
std,157.660387,177.315865,62.384721,191.096774,89.000144,143.669253,99.743264,231.663840,0.316880,182.615225,186.042622,124.274558,67.614425,136.312450,202.726660
min,0.000000,0.000000,0.000000,-122.000000,0.000000,-83.000000,0.000000,0.000000,0.000000,0.000000,-53.000000,-83.000000,0.000000,-193.000000,-83.000000
25%,0.000000,26.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.958333,0.208333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,97.000000,8.000000,26.000000,10.000000,0.000000,0.000000,37.875000,0.666666,15.000000,43.000000,0.000000,0.000000,0.000000,52.000000
75%,122.000000,276.000000,27.000000,253.000000,32.000000,40.000000,17.000000,187.958328,0.833333,197.000000,274.000000,13.000000,0.000000,2.000000,314.000000
max,640.000000,640.000000,819.000000,869.000000,819.000000,873.000000,843.000000,1707.791626,0.958333,876.000000,670.000000,648.000000,847.000000,878.000000,879.000000


In [202]:
find_num = 0.8
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
D.corr().style.applymap(draw_color, color='green')

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15
D1,1.000000,0.981311,0.280423,0.585714,0.055236,0.525691,0.204924,0.050165,0.007667,0.562145,0.592629,0.521104,0.370195,0.154799,0.638957
D2,0.981311,1.000000,0.266112,0.570017,0.238365,0.601299,0.454152,0.275304,0.042867,0.525967,0.563220,0.668833,0.394203,0.196670,0.618243
D3,0.280423,0.266112,1.000000,0.149038,0.707425,0.304947,0.818080,0.194211,0.006983,0.092362,0.055433,0.349025,0.136444,0.029219,0.141511
D4,0.585714,0.570017,0.149038,1.000000,0.313650,0.956966,0.574098,0.162900,0.041768,0.636928,0.657797,0.999999,0.383803,0.135268,0.751546
D5,0.055236,0.238365,0.707425,0.313650,1.000000,0.551310,0.986496,0.249365,0.031991,0.116784,0.058496,0.565222,0.134026,0.064274,0.188533
D6,0.525691,0.601299,0.304947,0.956966,0.551310,1.000000,0.506527,0.185532,0.040119,0.463143,nan,0.976834,0.419894,0.207166,0.700615
D7,0.204924,0.454152,0.818080,0.574098,0.986496,0.506527,1.000000,0.223624,0.025509,0.107557,nan,0.553886,0.123023,0.035528,0.285026
D8,0.050165,0.275304,0.194211,0.162900,0.249365,0.185532,0.223624,1.000000,0.066085,0.168607,nan,0.157520,0.521432,0.080991,0.226635
D9,0.007667,0.042867,0.006983,0.041768,0.031991,0.040119,0.025509,0.066085,1.000000,0.063333,nan,0.026858,0.024555,0.042368,0.054647
D10,0.562145,0.525967,0.092362,0.636928,0.116784,0.463143,0.107557,0.168607,0.063333,1.000000,0.609652,0.018188,0.277137,0.336933,0.712252


In [203]:
D.corr().mean()

D1     0.435397
D2     0.493104
D3     0.302081
D4     0.527966
D5     0.350745
D6     0.531432
D7     0.421316
D8     0.254455
D9     0.105339
D10    0.379399
D11    0.537778
D12    0.490865
D13    0.335360
D14    0.190720
D15    0.489209
dtype: float64

#### M1 - M9

In [204]:
# M1-M9
train_transaction.iloc[: , 46:55].head(20)

,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,T,T,T,M2,F,T,NaN,NaN,NaN
1,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN
2,T,T,T,M0,F,F,F,F,F
3,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,T,T,T,M1,F,T,NaN,NaN,NaN
6,T,T,T,M0,F,F,T,T,T
7,NaN,NaN,NaN,M0,F,F,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,T,T,T,M0,T,T,NaN,NaN,NaN


In [205]:
M = train_transaction.iloc[: , 46:55]
M.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,T,T,T,M2,F,T,NaN,NaN,NaN
1,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN
2,T,T,T,M0,F,F,F,F,F
3,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
M.describe()

,M1,M2,M3,M4,M5,M6,M7,M8,M9
count,319440,319440,319440,309096,240058,421180,244275,244288,244288
unique,2,2,2,3,2,2,2,2,2
top,T,T,T,M0,F,F,F,F,T
freq,319415,285468,251731,196405,132491,227856,211374,155251,205656


In [207]:
# map 함수 사용
M_copy = M.copy()
match = {"F": 0, "T": 1}

for i in range(9):
    M_copy.iloc[:, i] = M_copy.iloc[:, i].map(match)

M_copy

,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,1.0,1.0,1.0,NaN,0.0,1.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
2,1.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
590535,1.0,1.0,1.0,NaN,1.0,0.0,0.0,0.0,1.0
590536,1.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0
590537,1.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN
590538,1.0,1.0,1.0,NaN,0.0,1.0,NaN,NaN,NaN


In [208]:
find_num = 0.6
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
M_copy.corr().style.applymap(draw_color, color='green')

,M1,M2,M3,M4,M5,M6,M7,M8,M9
M1,1.000000,0.022202,0.017058,nan,-0.000482,0.002873,0.001790,0.003438,0.008035
M2,0.022202,1.000000,0.665161,nan,0.017961,0.037023,0.098407,0.177217,0.408110
M3,0.017058,0.665161,1.000000,nan,-0.011867,0.021657,0.060444,0.098663,0.232560
M4,nan,nan,nan,nan,nan,nan,nan,nan,nan
M5,-0.000482,0.017961,-0.011867,nan,1.000000,-0.035068,0.007124,0.015034,0.171504
M6,0.002873,0.037023,0.021657,nan,-0.035068,1.000000,0.082266,0.084646,0.090393
M7,0.001790,0.098407,0.060444,nan,0.007124,0.082266,1.000000,0.493290,0.169445
M8,0.003438,0.177217,0.098663,nan,0.015034,0.084646,0.493290,1.000000,0.323376
M9,0.008035,0.408110,0.232560,nan,0.171504,0.090393,0.169445,0.323376,1.000000


#### Vxxx

In [209]:
# Vxxx
train_transaction.iloc[: , 55:].head(20)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
V = train_transaction.iloc[: , 55:]
V.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
V.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,0.999945,1.045204,1.078075,0.846456,0.876991,1.045686,1.072870,1.027704,1.041529,0.463915,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,0.007390,0.240133,0.320890,0.440053,0.475902,0.239385,0.304779,0.186069,0.226339,0.521522,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.000000,8.000000,9.000000,6.000000,6.000000,9.000000,9.000000,8.000000,8.000000,4.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


### isFraud (merged)

In [212]:
F_merged = df_train[df_train['isFraud']==1]
F_merged.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
203,2987203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,2987240,0.0,169947.0,0.0,0.0,3.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,2987243,0.0,222455.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,2987245,0.0,271870.0,0.0,0.0,3.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,2987288,-20.0,258138.0,NaN,NaN,0.0,-1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
F_merged.describe().iloc[:, 375:]

,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,20661.000000,20617.000000,20617.000000,20617.000000,20661.000000,20661.000000,20661.000000,20661.000000,20661.000000,20661.000000,...,3686.000000,3686.000000,3686.000000,3686.000000,3686.000000,3686.000000,3686.000000,3686.000000,3686.000000,3686.000000
mean,73.144119,42.250735,78.456388,56.447212,74.526581,352.323304,176.781527,18.478514,54.409545,31.691673,...,0.313619,90.384699,164.859468,127.633749,9.299240,49.394466,25.483993,37.742268,53.242268,45.496473
std,221.490399,139.558705,258.446060,187.113982,336.081637,2912.608406,894.133506,123.652990,307.351165,177.831364,...,0.960475,227.643019,436.287131,339.856944,51.502023,201.848261,117.938993,142.505208,190.250383,175.966280
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,30.950001,0.000000,0.000000,0.000000,0.000000,65.004501,28.397400,0.000000,0.000000,0.000000,...,0.000000,75.000000,150.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3981.000000,2652.000000,3981.000000,3981.000000,11848.000000,82130.953125,18123.957031,4528.899902,5794.500000,4528.899902,...,13.000000,2600.000000,4700.000000,4285.000000,635.000000,2430.000000,1605.000000,2200.000000,2400.000000,2400.000000


In [214]:
F_merged.describe(include=['O'])

,id_12,id_15,id_16,id_23,id_27,id_28,id_29,id_30,id_31,id_33,...,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
count,11318,11222,10152,426,426,11218,11218,3442,11171,3363,...,11227,6342,6342,6342,15436,7569,8692,4817,4817,4817
unique,2,3,2,3,1,2,2,59,99,59,...,32,1,2,2,3,2,2,2,2,2
top,NotFound,Found,Found,IP_PROXY:TRANSPARENT,Found,Found,Found,Windows 10,chrome 63.0,1366x768,...,gmail.com,T,T,T,M0,T,F,F,F,T
freq,10049,7118,7112,245,426,7833,7827,657,1503,493,...,6811,6342,5158,4293,7198,4055,5397,4089,3373,3658


In [215]:
train_transaction.describe(include=['O'])

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
count,590540,588963,588969,496084,137291,319440,319440,319440,309096,240058,421180,244275,244288,244288
unique,5,4,4,59,60,2,2,2,3,2,2,2,2,2
top,W,visa,debit,gmail.com,gmail.com,T,T,T,M0,F,F,F,F,T
freq,439670,384767,439938,228355,57147,319415,285468,251731,196405,132491,227856,211374,155251,205656


In [216]:
print('ProductCD :', round(8969/20663 * 100, 2))
print('card4 :', round(13373/20622 * 100, 2))
print('card6 :', round(10674/20624 * 100, 2))
print('P_emaildomain :', round(9943/17873 * 100, 2))
print('R_emaildomain :', round(6811/11227 * 100, 2))
print('M1 :', round(6342/6342 * 100, 2))
print('M2 :', round(5158/6342 * 100, 2))
print('M3 :', round(4293/6342 * 100, 2))
print('M4 :', round(7198/15436 * 100, 2))
print('M5 :', round(4055/7569 * 100, 2))
print('M6 :', round(5397/8692 * 100, 2))
print('M7 :', round(4089/4817 * 100, 2))
print('M8 :', round(3373/4817 * 100, 2))
print('M9 :', round(3658/4817 * 100, 2))

ProductCD : 43.41
card4 : 64.85
card6 : 51.76
P_emaildomain : 55.63
R_emaildomain : 60.67
M1 : 100.0
M2 : 81.33
M3 : 67.69
M4 : 46.63
M5 : 53.57
M6 : 62.09
M7 : 84.89
M8 : 70.02
M9 : 75.94


In [217]:
F_merged.card5.value_counts()

226.0    8753
224.0    3152
102.0    1832
137.0    1725
138.0    1682
195.0     681
166.0     630
219.0     571
117.0     354
162.0     301
126.0     105
197.0      86
223.0      83
202.0      58
147.0      47
198.0      34
229.0      32
141.0      32
203.0      28
150.0      23
146.0      23
190.0      20
236.0      20
185.0      19
100.0      18
144.0      16
118.0      15
194.0      13
228.0      13
142.0      12
129.0       9
133.0       8
135.0       8
132.0       7
119.0       6
149.0       5
212.0       5
107.0       4
143.0       4
232.0       4
139.0       3
183.0       3
159.0       2
237.0       2
191.0       1
134.0       1
167.0       1
131.0       1
199.0       1
Name: card5, dtype: int64

In [218]:
F_merged.ProductCD.unique()

array(['W', 'C', 'R', 'S', 'H'], dtype=object)

In [219]:
F_merged.card4.unique()

array(['visa', 'mastercard', 'discover', 'american express', nan],
      dtype=object)

In [220]:
F_merged.card4.isna().sum()

41

In [221]:
F_merged.P_emaildomain.unique()

array(['aol.com', 'hotmail.com', 'outlook.com', 'gmail.com', 'yahoo.com',
       nan, 'anonymous.com', 'icloud.com', 'frontiernet.net', 'me.com',
       'ymail.com', 'att.net', 'sbcglobal.net', 'comcast.net', 'live.com',
       'mail.com', 'charter.net', 'gmail', 'msn.com', 'cox.net',
       'verizon.net', 'protonmail.com', 'bellsouth.net', 'live.com.mx',
       'cableone.net', 'yahoo.com.mx', 'optonline.net', 'aim.com',
       'prodigy.net.mx', 'outlook.es', 'hotmail.es', 'suddenlink.net',
       'earthlink.net', 'yahoo.fr', 'rocketmail.com', 'roadrunner.com',
       'embarqmail.com', 'yahoo.es', 'mac.com', 'frontier.com',
       'sc.rr.com', 'netzero.net', 'juno.com'], dtype=object)

In [222]:
F_merged.P_emaildomain.value_counts()

gmail.com          9943
hotmail.com        2396
yahoo.com          2297
anonymous.com       859
aol.com             617
outlook.com         482
comcast.net         246
icloud.com          197
mail.com            106
msn.com              90
live.com             84
outlook.es           57
bellsouth.net        53
ymail.com            50
live.com.mx          41
aim.com              40
protonmail.com       31
att.net              30
cox.net              29
me.com               27
charter.net          25
verizon.net          22
hotmail.es           20
optonline.net        17
yahoo.com.mx         16
mac.com              14
sbcglobal.net        12
earthlink.net        11
gmail                11
embarqmail.com        9
frontier.com          8
juno.com              6
yahoo.fr              5
frontiernet.net       5
suddenlink.net        4
roadrunner.com        3
cableone.net          3
rocketmail.com        2
yahoo.es              2
sc.rr.com             1
netzero.net           1
prodigy.net.mx  

In [223]:
F_merged.M4.unique()

array(['M0', 'M2', nan, 'M1'], dtype=object)

#### Correlation

In [224]:
F_corr1 =  F_merged.iloc[:, [0,1,2,3,4,5,29,30,31,32,33,34,35,40,41,42,43,45,46,47,48,49,50,51,52]].corr()
F_corr1

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_32,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2
TransactionID,1.000000,-0.025584,0.107329,0.019858,-0.006009,-0.067696,-0.090804,NaN,0.998281,0.040154,0.035487,-0.013245,0.013920,0.013078,0.076663,0.147490
id_01,-0.025584,1.000000,-0.094098,0.012276,0.009980,0.076740,0.014832,NaN,-0.025919,0.078457,-0.053637,-0.007347,-0.057141,0.016572,-0.032658,0.149745
id_02,0.107329,-0.094098,1.000000,-0.026392,0.006716,-0.033451,0.177541,NaN,0.105176,0.034969,0.025883,-0.034663,0.143979,-0.025424,0.010331,0.037875
id_03,0.019858,0.012276,-0.026392,1.000000,0.474197,0.045143,-0.105846,NaN,0.019887,-0.015629,-0.004629,0.014130,0.053952,-0.039463,-0.045994,-0.027752
id_04,-0.006009,0.009980,0.006716,0.474197,1.000000,0.033780,-0.009747,NaN,-0.007774,0.000227,-0.018021,0.009123,0.002131,-0.024147,-0.036278,0.039509
id_05,-0.067696,0.076740,-0.033451,0.045143,0.033780,1.000000,0.286504,NaN,-0.068904,0.251403,-0.004881,0.082522,-0.266643,0.067763,0.014570,0.136605
id_32,-0.090804,0.014832,0.177541,-0.105846,-0.009747,0.286504,1.000000,NaN,-0.089424,0.226341,0.029045,0.001117,0.057118,-0.039628,0.041672,0.027539
isFraud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TransactionDT,0.998281,-0.025919,0.105176,0.019887,-0.007774,-0.068904,-0.089424,NaN,1.000000,0.039755,0.035200,-0.013747,0.014511,0.014398,0.077384,0.135555
TransactionAmt,0.040154,0.078457,0.034969,-0.015629,0.000227,0.251403,0.226341,NaN,0.039755,1.000000,0.018249,0.076691,-0.306987,0.048047,-0.038431,0.082265


In [225]:
find_num = 0.4
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
F_corr1.style.applymap(draw_color, color='blue')

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_32,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2
TransactionID,1.000000,-0.025584,0.107329,0.019858,-0.006009,-0.067696,-0.090804,nan,0.998281,0.040154,0.035487,-0.013245,0.013920,0.013078,0.076663,0.147490
id_01,-0.025584,1.000000,-0.094098,0.012276,0.009980,0.076740,0.014832,nan,-0.025919,0.078457,-0.053637,-0.007347,-0.057141,0.016572,-0.032658,0.149745
id_02,0.107329,-0.094098,1.000000,-0.026392,0.006716,-0.033451,0.177541,nan,0.105176,0.034969,0.025883,-0.034663,0.143979,-0.025424,0.010331,0.037875
id_03,0.019858,0.012276,-0.026392,1.000000,0.474197,0.045143,-0.105846,nan,0.019887,-0.015629,-0.004629,0.014130,0.053952,-0.039463,-0.045994,-0.027752
id_04,-0.006009,0.009980,0.006716,0.474197,1.000000,0.033780,-0.009747,nan,-0.007774,0.000227,-0.018021,0.009123,0.002131,-0.024147,-0.036278,0.039509
id_05,-0.067696,0.076740,-0.033451,0.045143,0.033780,1.000000,0.286504,nan,-0.068904,0.251403,-0.004881,0.082522,-0.266643,0.067763,0.014570,0.136605
id_32,-0.090804,0.014832,0.177541,-0.105846,-0.009747,0.286504,1.000000,nan,-0.089424,0.226341,0.029045,0.001117,0.057118,-0.039628,0.041672,0.027539
isFraud,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
TransactionDT,0.998281,-0.025919,0.105176,0.019887,-0.007774,-0.068904,-0.089424,nan,1.000000,0.039755,0.035200,-0.013747,0.014511,0.014398,0.077384,0.135555
TransactionAmt,0.040154,0.078457,0.034969,-0.015629,0.000227,0.251403,0.226341,nan,0.039755,1.000000,0.018249,0.076691,-0.306987,0.048047,-0.038431,0.082265


##### isFraud identity part correlation (encoded)

In [226]:
F_copy = F_merged.copy()

In [227]:
F_copy.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=434)

In [228]:
IDcolumns = ['id_12', 'id_15','id_16','id_23','id_27','id_28','id_29','id_30','id_31','id_33','id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

In [229]:
from sklearn.preprocessing import LabelEncoder    # LabelEncoder를 불러오기
encoder = LabelEncoder()      # encoder라는 변수를 선언

for col in IDcolumns:
    encoder.fit(F_copy[col]) # title 열에 맞게 인코딩
    F_copy[col] = encoder.transform(F_copy[col]) # 인코딩 결과를 실제 행에 적용
    F_copy.head(10)

In [230]:
encoder.inverse_transform([0, 1, 2, 3, 4])

array(['0PM92', '4013M Build/KOT49H', '4047G Build/NRD90M',
       '5010G Build/MRA58K', '5011A Build/NRD90M'], dtype=object)

In [231]:
F_corr2 =  F_copy.iloc[:, :41].corr()
find_num = 0.8
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
F_corr2.style.applymap(draw_color, color='coral')

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,1.000000,-0.025584,0.107329,0.019858,-0.006009,-0.067696,0.049442,-0.228005,0.012366,-0.009758,-0.006921,0.021434,0.011814,0.079642,0.113814,0.009225,0.005648,0.074556,0.107906,-0.009864,0.043400,0.007524,-0.040847,-0.006544,0.215604,0.052401,0.094798,0.001315,0.006796,0.006573,0.064822,0.038864,-0.090804,0.063956,0.081492,-0.012270,0.015272,0.015041,-0.129919,0.014453,-0.046855
id_01,-0.025584,1.000000,-0.094098,0.012276,0.009980,0.076740,0.169529,0.112441,-0.209951,0.004731,0.016957,-0.024771,-0.084051,0.072938,-0.225242,-0.124285,-0.128888,-0.142749,-0.016902,-0.075479,-0.098960,-0.089367,-0.119658,0.145756,-0.009874,0.278963,0.046116,0.158659,-0.068419,-0.067752,-0.102010,0.049525,0.014832,-0.132696,-0.149703,0.145969,0.033982,0.097665,0.094785,-0.125543,0.031758
id_02,0.107329,-0.094098,1.000000,-0.026392,0.006716,-0.033451,-0.056711,0.121112,-0.029024,-0.031119,0.005016,0.066488,0.203516,-0.060249,-0.085105,0.069196,0.068651,0.163693,0.074882,-0.016872,0.099123,-0.043200,0.062396,0.026524,-0.014071,-0.029913,-0.051795,0.019842,0.057222,0.056553,0.170979,0.055472,0.177541,0.156099,0.113017,-0.169762,-0.071942,-0.129468,-0.291652,0.315006,-0.191469
id_03,0.019858,0.012276,-0.026392,1.000000,0.474197,0.045143,0.055808,0.056815,0.198560,0.861850,0.396559,nan,0.025495,0.022860,-0.084044,nan,nan,0.037943,-0.005512,-0.044159,-0.001859,0.208206,0.142426,-0.003352,-0.008358,0.093637,0.254158,0.008572,nan,-0.001349,0.014183,-0.009011,-0.105846,0.023989,0.030354,-0.023421,-0.000044,0.028813,-0.034158,-0.097659,0.085387
id_04,-0.006009,0.009980,0.006716,0.474197,1.000000,0.033780,0.030788,0.116051,0.070266,0.443666,0.883572,nan,-0.030001,-0.014983,-0.259382,nan,nan,-0.016120,0.037275,-0.012655,-0.027562,0.132299,-0.084284,0.029394,-0.312710,-0.093481,0.022422,0.032049,nan,0.001620,-0.029694,0.049027,-0.009747,-0.035858,-0.044783,0.042519,0.021694,-0.034605,-0.005318,-0.037833,0.044207
id_05,-0.067696,0.076740,-0.033451,0.045143,0.033780,1.000000,-0.165921,0.122096,0.054831,0.077468,0.014469,0.004004,-0.114878,-0.054513,-0.081352,-0.022534,-0.022685,-0.306897,-0.156849,-0.023797,-0.039353,-0.040928,-0.101217,-0.177135,-0.167884,-0.068106,0.092401,-0.190542,0.009561,0.009374,-0.213799,0.203319,0.286504,-0.275131,-0.363199,0.324329,-0.002808,-0.138073,0.093050,0.090259,0.090155
id_06,0.049442,0.169529,-0.056711,0.055808,0.030788,-0.165921,1.000000,-0.132295,0.402369,0.054877,0.025877,-0.026605,-0.022532,0.046642,0.011079,-0.034965,-0.034628,0.042074,0.155199,-0.050104,-0.125842,0.184166,0.174545,0.196310,0.007829,-0.072805,-0.098117,0.223022,-0.053269,-0.052769,0.006687,-0.033852,-0.232680,0.009097,0.061953,-0.031538,0.043964,0.088105,-0.003073,-0.196725,0.044696
id_07,-0.228005,0.112441,0.121112,0.056815,0.116051,0.122096,-0.132295,1.000000,-0.030235,0.043515,0.117162,-0.077714,0.018486,-0.298079,-0.288755,-0.139971,-0.139971,-0.160674,-0.297844,-0.004060,-0.105942,-0.428972,-0.414192,0.578041,-0.183079,0.007479,-0.157963,nan,-0.070876,-0.070876,-0.244653,0.157909,0.382269,-0.090747,-0.387308,0.433046,-0.171754,-0.032978,-0.072166,0.305604,0.073063
id_08,0.012366,-0.209951,-0.029024,0.198560,0.070266,0.054831,0.402369,-0.030235,1.000000,0.177127,0.068022,-0.011921,-0.106516,-0.046870,-0.005149,0.002733,0.002733,-0.135467,-0.171983,0.157409,0.004699,0.058740,-0.131202,-0.027432,-0.039935,-0.034882,-0.151644,nan,-0.013344,-0.013344,-0.086770,-0.041997,-0.211247,-0.150203,-0.067326,0.077663,0.055955,0.136298,0.065807,-0.292780,0.183476
id_09,-0.009758,0.004731,-0.031119,0.861850,0.443666,0.077468,0.054877,0.043515,0.177127,1.000000,0.423941,-0.010499,0.016954,0.025589,-0.083708,0.000475,0.000422,0.012428,0.042012,-0.009127,-0.014982,0.226258,0.121373,-0.009043,0.0

##### isFraud transaction part correlation (encoded)

In [232]:
TAcolumns = ['ProductCD', 'card4','card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']

In [233]:
from sklearn.preprocessing import LabelEncoder    # LabelEncoder를 불러오기
encoder = LabelEncoder()      # encoder라는 변수를 선언

for tacol in TAcolumns:
    encoder.fit(F_copy[tacol]) # title 열에 맞게 인코딩
    F_copy[tacol] = encoder.transform(F_copy[tacol]) # 인코딩 결과를 실제 행에 적용
    F_copy.head(10)

In [234]:
F_corr3 =  F_copy.iloc[:, 42:71].corr()
find_num = 0.8
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
F_corr3.style.applymap(draw_color, color='skyblue')

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
TransactionDT,1.000000,0.039755,0.020181,0.035200,-0.013747,0.014511,0.022868,0.014398,0.053577,0.077384,0.135555,-0.026821,-0.036284,0.015086,0.016352,-0.086404,-0.086979,0.017304,-0.085416,0.034096,-0.081349,-0.107599,-0.100725,0.048255,-0.107801,-0.088675,-0.106451,-0.090701,-0.080579
TransactionAmt,0.039755,1.000000,0.335833,0.018249,0.076691,-0.306987,-0.041100,0.048047,-0.071440,-0.038431,0.082265,0.023474,0.013582,0.026425,0.281028,-0.038292,-0.046166,-0.003969,-0.026207,0.003729,-0.023327,-0.045780,-0.041985,0.025226,-0.038245,-0.033503,-0.050382,-0.027955,-0.016254
ProductCD,0.020181,0.335833,1.000000,0.056180,0.067350,-0.788312,0.029164,0.246940,0.225634,-0.016688,0.385573,nan,0.019395,0.316765,0.854860,-0.120832,-0.139861,-0.005490,-0.106612,0.115025,-0.091146,-0.115765,-0.121128,0.213382,-0.110410,-0.110196,-0.126586,-0.033782,-0.064664
card1,0.035200,0.018249,0.056180,1.000000,-0.000138,-0.060676,0.095728,-0.057372,0.050773,0.011375,0.026995,0.049152,0.009946,-0.006928,0.042666,0.005537,0.003476,0.008143,0.008546,-0.017264,0.007509,0.006448,0.005809,-0.003118,0.007507,0.004807,0.004014,0.005925,0.009036
card2,-0.013747,0.076691,0.067350,-0.000138,1.000000,-0.115585,0.003214,0.149751,-0.133794,0.114190,-0.028506,-0.059750,-0.003093,0.007242,0.039837,-0.011687,-0.014059,0.000613,-0.002543,-0.023779,-0.002990,-0.012486,-0.007879,-0.011601,-0.008840,-0.005870,-0.012694,-0.010282,-0.001317
card3,0.014511,-0.306987,-0.788312,-0.060676,-0.115585,1.000000,-0.000318,-0.269582,-0.111408,-0.024789,-0.513432,-0.015791,-0.051312,-0.216726,-0.569767,0.114085,0.132244,-0.011009,0.092431,-0.076875,0.081645,0.120590,0.117398,-0.142207,0.106535,0.103306,0.132068,0.045469,0.057981
card4,0.022868,-0.041100,0.029164,0.095728,0.003214,-0.000318,1.000000,0.218035,0.210460,0.075757,0.005206,-0.025674,-0.005739,0.004474,0.050119,-0.013195,-0.013125,-0.000082,-0.016515,0.012469,-0.014535,-0.009479,-0.012865,0.036302,-0.012788,-0.013036,-0.008396,-0.003971,-0.013427
card5,0.014398,0.048047,0.246940,-0.057372,0.149751,-0.269582,0.218035,1.000000,0.188383,0.095865,0.062862,-0.021050,-0.002390,0.099857,0.197793,-0.040452,-0.043249,0.011342,-0.039004,0.046887,-0.033696,-0.039168,-0.041262,0.077786,-0.037839,-0.038875,-0.042534,-0.011527,-0.027248
card6,0.053577,-0.071440,0.225634,0.050773,-0.133794,-0.111408,0.210460,0.188383,1.000000,0.037740,0.078069,0.036220,0.035035,0.117476,0.235033,-0.012077,-0.016320,0.002462,-0.015302,0.052308,-0.010359,-0.003172,-0.009114,0.074831,-0.006174,-0.013324,-0.007094,0.021400,-0.003819
addr1,0.077384,-0.038431,-0.016688,0.011375,0.114190,-0.024789,0.075757,0.095865,0.037740,1.000000,0.023200,-0.030226,0.018812,-0.012068,-0.031130,-0.007339,-0.004187,-0.005873,-0.007624,0.017713,-0.005192,-0.001762,-0.005441,0.014928,-0.004413,-0.004509,-0.000247,0.003514,-0.003587


In [235]:
F_corr4 =  F_copy.iloc[:, 71:95].corr()
find_num = 0.8
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
F_corr4.style.applymap(draw_color, color='skyblue')

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9
D1,1.000000,0.961212,0.309173,0.489803,0.059454,0.237904,0.073245,0.023608,0.038972,0.465348,0.511824,0.197076,0.218829,0.038318,0.575330,-0.177989,-0.164333,-0.161521,-0.111206,-0.165064,-0.227052,-0.112867,-0.107138,-0.109513
D2,0.961212,1.000000,0.281805,0.565249,0.198047,0.489239,0.302400,0.102140,0.069232,0.485862,0.536531,0.428263,0.398812,0.081636,0.626199,-0.037649,-0.032016,-0.030157,0.081007,0.023129,-0.065784,-0.010864,-0.004110,-0.005773
D3,0.309173,0.281805,1.000000,0.052878,0.311656,0.104370,0.211284,0.035537,0.004542,0.021457,0.093223,0.113145,0.029504,0.023885,0.053910,0.089433,0.083705,0.079960,0.092320,0.124962,0.132486,0.068952,0.074046,0.068555
D4,0.489803,0.565249,0.052878,1.000000,0.315773,0.995072,0.433510,0.075108,0.031680,0.538678,0.522901,1.000000,0.221791,0.100060,0.628311,-0.165134,-0.154561,-0.157514,-0.093363,-0.150174,-0.197384,-0.111217,-0.108785,-0.118696
D5,0.059454,0.198047,0.311656,0.315773,1.000000,0.435715,0.999735,0.101627,-0.015985,0.110677,0.096771,0.436898,0.081757,0.030635,0.156135,-0.032423,-0.036621,-0.035137,0.012204,-0.023834,-0.039265,-0.036055,-0.031125,-0.042480
D6,0.237904,0.489239,0.104370,0.995072,0.435715,1.000000,0.415319,0.080782,0.028583,0.170904,nan,0.995159,0.275048,0.123375,0.325811,nan,nan,nan,0.031505,nan,nan,nan,nan,nan
D7,0.073245,0.302400,0.211284,0.433510,0.999735,0.415319,1.000000,0.109052,-0.014480,0.001420,nan,0.433424,0.076023,0.034027,0.070368,nan,nan,nan,0.024222,nan,nan,nan,nan,nan
D8,0.023608,0.102140,0.035537,0.075108,0.101627,0.080782,0.109052,1.000000,0.060816,0.115514,nan,0.064963,0.577682,-0.011362,0.078518,nan,nan,nan,0.092288,nan,nan,nan,nan,nan
D9,0.038972,0.069232,0.004542,0.031680,-0.015985,0.028583,-0.014480,0.060816,1.000000,0.028330,nan,0.028664,0.028208,-0.001164,0.073790,nan,nan,nan,0.037499,nan,nan,nan,nan,nan
D10,0.465348,0.485862,0.021457,0.538678,0.110677,0.170904,0.001420,0.115514,0.028330,1.000000,0.492396,0.005705,0.091624,-0.002474,0.669706,-0.258598,-0.241895,-0.252896,-0.128461,-0.237142,-0.289136,-0.183173,-0.169741,-0.169188


In [236]:
F_corr5 =  F_copy.iloc[:, 95:150].corr()
find_num = 0.8
def draw_color(x,color):
    if (x >= find_num or x <= (-find_num)) & (x != 1):
        color = f'background-color:{color}'
        return color
    else:
        return ''
    
F_corr5.style.applymap(draw_color, color='skyblue')

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55
V1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
V2,nan,1.000000,0.695381,0.387834,0.358968,0.645635,0.478703,0.766978,0.599655,0.139531,0.138016,0.028081,0.028722,nan,nan,nan,nan,nan,-0.015987,0.001608,nan,nan,-0.018087,-0.025380,0.016623,0.034012,nan,nan,-0.027858,-0.026309,nan,nan,0.031028,0.044721,0.054634,0.069683,0.299897,0.202813,nan,nan,nan,nan,nan,-0.024014,-0.020581,0.101366,0.103063,-0.021203,-0.019238,nan,0.127060,0.141730,-0.001553,0.054440,0.280350
V3,nan,0.695381,1.000000,0.252925,0.489069,0.420593,0.550574,0.501762,0.668416,0.036845,0.050983,0.032864,0.049433,nan,nan,nan,nan,nan,0.044454,0.080161,nan,nan,-0.007389,0.014580,0.005085,0.082872,nan,nan,-0.073790,-0.071827,nan,nan,0.092713,0.083173,-0.000737,0.007155,0.263088,0.319123,nan,nan,nan,nan,nan,0.127897,0.167959,0.075822,0.151757,-0.076800,-0.069761,nan,0.090799,0.137457,0.073082,0.098681,0.272229
V4,nan,0.387834,0.252925,1.000000,0.766374,0.349130,0.237208,0.418573,0.291454,-0.017959,-0.020867,0.025968,0.040195,nan,nan,nan,nan,nan,0.445546,0.368202,nan,nan,0.030010,0.018954,0.036892,0.031121,nan,nan,-0.083361,-0.098352,nan,nan,0.069047,0.072167,0.109832,0.128435,0.072172,-0.034689,nan,nan,nan,nan,nan,-0.084854,-0.121863,-0.000151,-0.030746,-0.023407,-0.024784,nan,0.047872,0.067638,-0.024929,0.039138,0.058489
V5,nan,0.358968,0.489069,0.766374,1.000000,0.268239,0.319920,0.331749,0.476552,-0.076181,-0.068945,0.025121,0.042033,nan,nan,nan,nan,nan,0.398221,0.476064,nan,nan,0.016885,0.051403,0.035213,0.136332,nan,nan,-0.127737,-0.136828,nan,nan,0.094434,0.114244,0.086741,0.091535,0.155288,0.091544,nan,nan,nan,nan,nan,0.034191,0.010231,0.001443,0.099849,-0.092450,-0.088516,nan,0.108540,0.176318,0.061906,0.096196,0.142534
V6,nan,0.645635,0.420593,0.349130,0.268239,1.000000,0.734825,0.707374,0.525381,0.197547,0.194508,0.007759,0.023458,nan,nan,nan,nan,nan,-0.022009,-0.031855,nan,nan,0.277773,0.222587,0.017859,0.002288,nan,nan,0.009821,0.007272,nan,nan,0.032717,0.025530,0.069150,0.094066,-0.003291,0.010709,nan,nan,nan,nan,nan,0.233658,0.153206,-0.011739,-0.033854,0.013352,0.014556,nan,0.017393,0.039831,-0.007648,0.056403,-0.007770
V7,nan,0.478703,0.550574,0.237208,0.319920,0.734825,1.000000,0.514432,0.587889,0.120221,0.131640,0.006609,0.025743,nan,nan,nan,nan,nan,0.013137,0.033788,nan,nan,0.233143,0.371781,0.001500,0.057214,nan,nan,-0.010886,-0.014695,nan,nan,0.008594,0.005974,0.041262,0.050633,0.051918,0.149808,nan,nan,nan,nan,nan,0.272630,0.433502,-0.007425,0.068514,-0.017429,-0.016175,nan,-0.008843,0.027656,0.027989,0.061619,-0.013926
V8,nan,0.766978,0.501762,0.418573,0.331749,0.707374,0.514432,1.000000,0.742465,0.155937,0.142929,0.000829,0.023559,nan,nan,nan,nan,nan,-0.017646,-0.031622,nan,nan,0.010257,0.017970,0.092901,0.080631,nan,nan,-0.037426,-0.039257,nan,nan,0.048764,0.038377,0.060010,0.078395,0.119137,0.075069,nan,nan,nan,nan,nan,0.051033,0.039992,0.202131,0.124873,-0.037341,-0.034831,nan,0.059841,0.047762,-0.029139,0.033714,0.110487
V9,nan,0.599655,0.668416,0.291454,0.476552,0.525381,0.587889,0.742465,1.000000,0.063565,0.071258,0.016055,0.030554,nan,nan,nan,nan,nan,0.035945,0.096113,nan,nan,0.003700,0.040849,0.105160,0.215901,nan,nan,-0.070138,-0.072185,nan,nan,0.057576,0.063626,-0.015077,-0.001165,0.117676,0.131891,nan,nan,nan,nan,nan,0.122977,0.171021,0.188564,0.252101,-0.082111,-0.078615,nan,0.078051,0.107380,0.066674,0.100323,0.174138
V10,nan,0.139531,0.036845,-0.017959,-0.076181,0.197547,0.120221,0.155937,0.063565,1.000000,0.972130,-0.046563,-0.098555,nan,nan,nan,nan,nan,-0.104230,-0.126909,nan,nan,-0.032925,-0.068319,-0.018227,-0.034637,nan,nan,0.860714,0.852381,nan,na

## 3. 전처리

### 결측치 처리 (아직 안함)

In [237]:
df_train.isna().sum()

TransactionID         0
id_01            446307
id_02            449668
id_03            524216
id_04            524216
                  ...  
V335             508189
V336             508189
V337             508189
V338             508189
V339             508189
Length: 434, dtype: int64

In [238]:
df_train.isna().any().sum()

# 결측치 있는 컬럼 : 412
# 결측치 없는 컬럼: 22

414

In [239]:
df_train.columns[df_train.nunique() == 1]


Index([], dtype='object')

In [240]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 434 entries, TransactionID to V339
dtypes: float64(399), int64(4), object(31)
memory usage: 1.9+ GB


In [241]:
df_train.isna().sum()

TransactionID         0
id_01            446307
id_02            449668
id_03            524216
id_04            524216
                  ...  
V335             508189
V336             508189
V337             508189
V338             508189
V339             508189
Length: 434, dtype: int64

### **feature engineering**  

1. ***feature 개수 축소***
2. Correlation (?)
3. feature importance / SequentialFeatureSelector 사용해보기
4. PCA (?)


#### **id_30 (75) 변수 줄이기 → 6개로 줄이기**
- Windows
- iOS
- Android
- Linux
- Mac
- other

In [242]:
df_train.id_30.value_counts()

Windows 10          21155
Windows 7           13110
iOS 11.2.1           3722
iOS 11.1.2           3699
Android 7.0          2871
                    ...  
func                   10
iOS 11.4.0              5
Mac OS X 10_13_5        4
Windows                 3
iOS 11.4.1              1
Name: id_30, Length: 75, dtype: int64

In [243]:
df_train.loc[df_train['id_30'].isin(['Windows 10', 'Windows 7', 'Windows 8.1', 'Windows Vista', 'Windows 8', 'Windows XP']),'id_30'] = 'Windows'

In [244]:
df_train.loc[df_train['id_30'].isin(['Android 7.0', 'Android 5.1.1', 'Android 7.1.1', 'Android 6.0.1', 'Android 8.0.0', 'Android 4.4.2', 'Android 8.1.0', 'Android 6.0', 'Android 7.1.2',
                                      'Android 5.0.2','Android 5.0',
                                       ]),'id_30'] = 'Android'

In [245]:
df_train.loc[df_train['id_30'].isin(['iOS 11.2.1', 
'iOS 11.2.5', 
'iOS 11.3.0', 
'iOS 11.1.2',
'iOS 11.2.2',
'iOS 11.2.6',
'iOS 10.3.3',
'iOS 11.2.0',
'iOS 11.1.1',
'iOS 11.1.0',
'iOS 11.0.3',
'iOS 10.3.2', 
'iOS 11.0.1',
'iOS 10.2.1', 
'iOS 10.3.1',
'iOS 11.0.2',
'iOS 11.0.0',
'iOS 10.2.0',
'iOS 9.3.5', 
'iOS 10.0.2',
'iOS 11.3.1',
'iOS 11.4.0',
'iOS 11.4.1',

'iOS 10.1.1',
]),'id_30'] = 'iOS'

In [246]:
df_train.loc[df_train['id_30'].isin(['Mac OS X 10_10_5',
'Mac OS X 10_13_3', 
'Mac OS X 10_12_6',      
'Mac OS X 10_9_5',      
'Mac OS X 10_13_4',      
'Mac OS X 10_11_6',     
'Mac OS X 10_13_2',      
'Mac OS X 10_13_1',      
'Mac OS X 10.12',         
'Mac OS X 10_12_1',       
'Mac OS X 10.10',         
'Mac OS X 10_11_5',       
'Mac OS X 10.11',         
'Mac OS X 10_12_3',       
'Mac OS X 10_12_5',       
'Mac OS X 10_11_4',       
'Mac OS X 10.13',
'Mac OS X 10_12_4',
'Mac OS X 10_7_5',
'Mac OS X 10_8_5',
'Mac OS X 10.9',
'Mac OS X 10_11_3',
'Mac OS X 10_6_8',
'Mac OS X 10_12',
'Mac OS X 10.6',
'Mac OS X 10_12_2',
'Mac OS X 10_13_5',
]),'id_30'] = 'Mac'


In [247]:
df_train.loc[df_train['id_30'].isin([
    'func',
]),'id_30'] = 'other'

In [248]:
df_train.id_30.value_counts()

Windows    36739
iOS        19782
Mac        13580
Android     6303
Linux       1136
other         25
Name: id_30, dtype: int64

#### **id_31 (130) 변수 줄이기 → 8개로 줄이기**
- chrome     7298
- safari     2384
- firefox     553
- edge        225
- samsung     196
- other       187
- ie          186
- opera       142

In [249]:
df_train.loc[df_train['id_31'].isin(['chrome 43.0 for android',
'chrome 49.0', 
'chrome 49.0 for android',
'chrome 50.0',
'chrome 50.0 for android',
'chrome 51.0',
'chrome 51.0 for android',
'chrome 52.0',
'chrome 52.0 for android',
'chrome 53.0',
'chrome 53.0 for android',
'chrome 54.0',
'chrome 54.0 for android',
'chrome 55.0',
'chrome 55.0 for android',
'chrome 56.0',
'chrome 56.0 for android',
'chrome 58.0',
'chrome 58.0 for android',
'chrome 59.0',
'chrome 59.0 for android',
'chrome 60.0',
'chrome 60.0 for android',
'chrome 61.0',
'chrome 61.0 for android',
'chrome 62.0',
'chrome 62.0 for android',
'chrome 63.0',
'chrome 63.0 for android',
'chrome 63.0 for ios',
'chrome 64.0',
'chrome 64.0 for android',
'chrome 65.0',
'chrome 65.0 for android',
'chrome 66.0',
'chrome 66.0 for android',
'chrome 67.0',
'chrome generic',
'chrome generic for android',
'chrome 66.0 for ios',
'chrome 46.0 for android',
'chrome 65.0 for ios',
'chrome 57.0 for android',
'chrome 62.0 for ios',
'chrome 57.0',
'chrome 64.0 for ios',
'chrome 67.0 for android',
'chrome 69.0',
]),'id_31'] = 'chrome'

In [250]:
df_train.loc[df_train['id_31'].isin(['safari generic',
'mobile safari 11.0', 
'mobile safari 10.0',
'mobile safari 9.0',
'mobile safari 8.0',
'mobile safari uiwebview',
'safari 10.0',
'safari 11.0',
'safari 9.0',
'mobile safari generic',
]),'id_31'] = 'safari'

In [251]:
df_train.loc[df_train['id_31'].isin(['firefox 47.0',
'firefox 48.0', 
'firefox 52.0',
'firefox 55.0',
'firefox 56.0',
'firefox 57.0',
'firefox 58.0',
'firefox 59.0',
'firefox 60.0',
'firefox generic',
'Mozilla/Firefox',
'firefox mobile 61.0',
]),'id_31'] = 'firefox'

In [252]:
df_train.loc[df_train['id_31'].isin(['ie 11.0 for desktop', 'ie 11.0 for tablet']),'id_31'] = 'ie'

In [253]:
df_train.loc[df_train['id_31'].isin(['edge 17.0',
                                     'edge 16.0',
                                     'edge 15.0',
                                     'edge 14.0',
                                     'edge 13.0',
                                     ]),'id_31'] = 'edge'

In [254]:
df_train.loc[df_train['id_31'].isin([
    'opera 49.0',
    'opera 51.0',
    'opera 52.0',
    'opera 53.0',
    'opera generic',    
                                     ]),'id_31'] = 'opera'

In [255]:
df_train.loc[df_train['id_31'].isin([
    'samsung browser 4.0',
    'samsung browser 4.2',
    'samsung browser 5.2',
    'samsung browser 5.4',
    'samsung browser 6.2',
    'samsung browser 6.4',
    'samsung browser 7.0',
    'samsung browser generic',
    'Samsung/SM-G531H',
    'Samsung/SM-G532M',
    'samsung browser 3.3',
    'Samsung/SCH',
]),'id_31'] = 'samsung'

In [256]:
df_train.loc[df_train['id_31'].isin([
    'icedragon',
    'comodo',
    'mobile',
    'google',
    'ZTE/Blade',
    'Lanix/Ilium',
    'android webview 4.0',
    'Generic/Android 7.0',
    'android browser 4.0',
    'Generic/Android',
    'google search application 48.0',
    'google search application 49.0',
    'Microsoft/Windows',
    'silk',
    'line',                                  
    'maxthon',                               
    'aol',                                   
    'palemoon',                              
    'puffin',                                
    'facebook',                              
    'waterfox',                              
    'Cherry',     
    'android',   
    'Inco/Minion',
    'cyberfox',   
    'chromium',   
    'M4Tel/M4',   
    'Nokia/Lumia',
    'seamonkey',  
    'BLU/Dash',   
    'iron',       
    'LG/K-200',
]),'id_31'] = 'other'

In [257]:
df_train.id_31.value_counts()

chrome     76059
safari     37281
ie          9733
firefox     7017
edge        6401
samsung     2247
other       1095
opera        449
Name: id_31, dtype: int64

#### **[🛶 진행중..] DeviceInfo (1786) 변수 줄이기 → __개로 줄이기.......**
-      
- 


In [258]:
df_train.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
rv:11.0         1901
               ...  
LGMS345            1
verykool           1
XT1072             1
LG-H931            1
0PJA2              1
Name: DeviceInfo, Length: 1786, dtype: int64

In [259]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Android') == True] = 'Android'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('ANDROID') == True] = 'Android'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('android') == True] = 'Android'
df_train.DeviceInfo.value_counts()

Windows            47722
iOS Device         19782
MacOS              12573
Trident/7.0         7440
rv:11.0             1901
                   ...  
SCH-I435               1
5045I                  1
SM-N920P               1
A97                    1
LG-H810/H81022f        1
Name: DeviceInfo, Length: 1779, dtype: int64

In [260]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Blade') == True] = 'BLADE'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('BLADE') == True] = 'BLADE'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('blade') == True] = 'BLADE'
df_train.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
rv:11.0         1901
               ...  
XT1072             1
LG-H931            1
SM-T807T           1
ASUS_Z017DA        1
ASUS_P00J          1
Name: DeviceInfo, Length: 1755, dtype: int64

In [261]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Hisense') == True] = 'HISENSE'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('HISENSE') == True] = 'HISENSE'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('hisense') == True] = 'HISENSE'
df_train.DeviceInfo.value_counts()# 

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
rv:11.0         1901
               ...  
HTC6500LVW         1
LG-H931            1
ASUS_Z017DA        1
O1                 1
SM-G900L           1
Name: DeviceInfo, Length: 1744, dtype: int64

In [262]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('HTC') == True] = 'HTC'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Htc') == True] = 'HTC'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('htc') == True] = 'HTC'
df_train.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
rv:11.0         1901
               ...  
verykool           1
XT1072             1
LG-H931            1
ASUS_Z017DA        1
ASUS_P00J          1
Name: DeviceInfo, Length: 1717, dtype: int64

In [263]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Huawei') == True] = 'HUAWEI'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('HUAWEI') == True] = 'HUAWEI'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('/Huawei') == True] = 'HUAWEI'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('hi6210sft') == True] = 'HUAWEI'
df_train.DeviceInfo.value_counts()

Windows          47722
iOS Device       19782
MacOS            12573
Trident/7.0       7440
HUAWEI            2605
                 ...  
LG-SP200             1
Alcatel_5056O        1
MAGNO                1
SM-S975L             1
LG-V496              1
Name: DeviceInfo, Length: 1652, dtype: int64

In [264]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Ilium') == True] = 'ILIUM'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('ILIUM') == True] = 'ILIUM'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('ilium') == True] = 'ILIUM'
df_train.DeviceInfo.value_counts()

Windows                          47722
iOS Device                       19782
MacOS                            12573
Trident/7.0                       7440
HUAWEI                            2605
                                 ...  
A97                                  1
A1-850                               1
LG-V500                              1
LGMS345                              1
SAMSUNG SM-A310M Build/LMY47X        1
Name: DeviceInfo, Length: 1636, dtype: int64

In [265]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Lenovo') == True] = 'LENOVO'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('LENOVO') == True] = 'LENOVO'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('lenovo') == True] = 'LENOVO'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('MOT-') == True] = 'LENOVO'
df_train.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
HUAWEI          2605
               ...  
A1-850             1
LG-V500            1
LGMS345            1
Tab2A7-10F         1
SCH-I435           1
Name: DeviceInfo, Length: 1617, dtype: int64

In [266]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('LG') == True] = 'LG'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('LG-') == True] = 'LG'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('VS') == True] = 'LG'
df_train.DeviceInfo.value_counts()

Windows              47722
iOS Device           19782
MacOS                12573
Trident/7.0           7440
LG                    2750
                     ...  
D6603                    1
SAMSUNG-SM-G930AZ        1
ATT                      1
Linux                    1
Z955A                    1
Name: DeviceInfo, Length: 1422, dtype: int64

In [267]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Linux') == True] = 'LINUX'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('LINUX') == True] = 'LINUX'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('linux') == True] = 'LINUX'
df_train.DeviceInfo.value_counts()

Windows              47722
iOS Device           19782
MacOS                12573
Trident/7.0           7440
LG                    2750
                     ...  
Coolpad                  1
D6603                    1
SAMSUNG-SM-G930AZ        1
ATT                      1
Z955A                    1
Name: DeviceInfo, Length: 1420, dtype: int64

In [268]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Moto') == True] = 'MOTO'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('MOTO') == True] = 'MOTO'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('mot') == True] = 'MOTO'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('XT') == True] = 'MOTO'
df_train.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
Trident/7.0     7440
MOTO            3510
               ...  
ASUS_Z017DA        1
O1                 1
Z831               1
GT-P3100           1
Z955A              1
Name: DeviceInfo, Length: 1270, dtype: int64

In [269]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Nexus') == True] = 'NEXUS'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('NEXUS') == True] = 'NEXUS'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('nexus') == True] = 'NEXUS'
df_train.DeviceInfo.value_counts()

Windows                       47722
iOS Device                    19782
MacOS                         12573
Trident/7.0                    7440
MOTO                           3510
                              ...  
GT-I8200N                         1
ZA409                             1
PLK-L01 Build/HONORPLK-L01        1
SM-G360F                          1
Z955A                             1
Name: DeviceInfo, Length: 1263, dtype: int64

In [270]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Pixel') == True] = 'PIXEL'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('pixel') == True] = 'PIXEL'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('PIXEL') == True] = 'PIXEL'
df_train.DeviceInfo.value_counts()

Windows                       47722
iOS Device                    19782
MacOS                         12573
Trident/7.0                    7440
MOTO                           3510
                              ...  
BND-L21 Build/HONORBND-L21        1
iris80                            1
Mobiistar_LAI_Yuna_X              1
verykools5034                     1
Z955A                             1
Name: DeviceInfo, Length: 1244, dtype: int64

In [271]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Redmi') == True] = 'REDMI'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('REDMI') == True] = 'REDMI'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('redmi') == True] = 'REDMI'
df_train.DeviceInfo.value_counts()

Windows                 47722
iOS Device              19782
MacOS                   12573
Trident/7.0              7440
MOTO                     3510
                        ...  
iris80                      1
Mobiistar_LAI_Yuna_X        1
verykools5034               1
A37f                        1
Z955A                       1
Name: DeviceInfo, Length: 1234, dtype: int64

In [272]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('rv:') == True] = 'rv:'
df_train.DeviceInfo.value_counts()

Windows                 47722
iOS Device              19782
MacOS                   12573
Trident/7.0              7440
rv:                      4385
                        ...  
iris80                      1
Mobiistar_LAI_Yuna_X        1
verykools5034               1
A37f                        1
Z955A                       1
Name: DeviceInfo, Length: 1204, dtype: int64

In [273]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('SAMSUNG') == True] = 'SAMSUNG'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('SM-') == True] = 'SAMSUNG'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Samsung') == True] = 'SAMSUNG'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('samsung') == True] = 'SAMSUNG'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('GT-') == True] = 'SAMSUNG'
df_train.DeviceInfo.value_counts()

Windows        47722
iOS Device     19782
MacOS          12573
SAMSUNG        12091
Trident/7.0     7440
               ...  
Stellar            1
SENS               1
VT0701A08          1
DUK-AL20           1
Z955A              1
Name: DeviceInfo, Length: 629, dtype: int64

In [274]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('F3') == True] = 'SONY'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('F5') == True] = 'SONY'
df_train.DeviceInfo.value_counts()

Windows               47722
iOS Device            19782
MacOS                 12573
SAMSUNG               12091
Trident/7.0            7440
                      ...  
NX785QC8G                 1
Dash                      1
916                       1
9203A Build/MRA58K        1
Z955A                     1
Name: DeviceInfo, Length: 608, dtype: int64

In [275]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('WINDOWS') == True] = 'Windows'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('Windows') == True] = 'Windows'
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('windows') == True] = 'Windows'
df_train.DeviceInfo.value_counts()

Windows        47775
iOS Device     19782
MacOS          12573
SAMSUNG        12091
Trident/7.0     7440
               ...  
Max                1
A1-850             1
A97                1
5045I              1
Z955A              1
Name: DeviceInfo, Length: 606, dtype: int64

In [276]:
df_train['DeviceInfo'][df_train['DeviceInfo'].str.contains('TA-') == True] = 'NOKIA'
df_train.DeviceInfo.value_counts()

Windows              47775
iOS Device           19782
MacOS                12573
SAMSUNG              12091
Trident/7.0           7440
                     ...  
ASUS                     1
QwestIE8                 1
SLAY                     1
Z959 Build/LMY47V        1
Z955A                    1
Name: DeviceInfo, Length: 584, dtype: int64

#### **P_emaildomain (59) 변수 줄이기 → 7개로 줄이기**
- gmail
- yahoo
- outlook
- anonymous
- aol
- icloud
- other

In [277]:
df_train.P_emaildomain.value_counts()

gmail.com           228355
yahoo.com           100934
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
yahoo.com.mx          1543
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
gmail                  496
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
hotmail.es             305
roadrunner.com         305
windstream.net         305
hotmail.fr             295
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
t

In [278]:
df_train.P_emaildomain.loc[df_train.P_emaildomain.str.contains('gmail') == True] = 'gmail'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo.com           100934
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
yahoo.com.mx          1543
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
windstream.net         305
hotmail.es             305
roadrunner.com         305
hotmail.fr             295
frontier.com           280
embarqmail.com         260
web.de                 240
twc.com                230
netzero.com            230
p

In [279]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('yahoo') == True] = 'yahoo'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook.com           5096
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
outlook.es             438
mac.com                436
juno.com               322
aim.com                315
hotmail.es             305
roadrunner.com         305
windstream.net         305
hotmail.fr             295
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
c

In [280]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('outlook') == True] = 'outlook'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
hotmail.com          45250
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
outlook               5534
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
hotmail.es             305
roadrunner.com         305
windstream.net         305
hotmail.fr             295
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
n

In [281]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('hotmail') == True] = 'outlook'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              51539
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
msn.com               4092
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
roadrunner.com         305
windstream.net         305
frontier.com           280
embarqmail.com         260
web.de                 240
twc.com                230
netzero.com            230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
s

In [282]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('msn') == True] = 'outlook'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              55631
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
att.net               4033
live.com              3041
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
live.com.mx            749
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
roadrunner.com         305
windstream.net         305
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
c

In [283]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('live') == True] = 'outlook'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              59477
anonymous.com        36998
aol.com              28289
comcast.net           7888
icloud.com            6267
att.net               4033
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
windstream.net         305
roadrunner.com         305
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
cfl.rr.com             172
sc.rr.com              164
c

In [284]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('anonymous') == True] = 'anonymous'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              59477
anonymous            36998
aol.com              28289
comcast.net           7888
icloud.com            6267
att.net               4033
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
windstream.net         305
roadrunner.com         305
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
cfl.rr.com             172
sc.rr.com              164
c

In [285]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('aol') == True] = 'aol'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              59477
anonymous            36998
aol                  28289
comcast.net           7888
icloud.com            6267
att.net               4033
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
windstream.net         305
roadrunner.com         305
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
cfl.rr.com             172
sc.rr.com              164
c

In [286]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('icloud') == True] = 'icloud'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              59477
anonymous            36998
aol                  28289
comcast.net           7888
icloud                6267
att.net               4033
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
rocketmail.com         664
mail.com               559
earthlink.net          514
mac.com                436
juno.com               322
aim.com                315
windstream.net         305
roadrunner.com         305
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
cfl.rr.com             172
sc.rr.com              164
c

In [287]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('mac') == True] = 'icloud'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              59477
anonymous            36998
aol                  28289
comcast.net           7888
icloud                6703
att.net               4033
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
me.com                1522
cox.net               1393
optonline.net         1011
charter.net            816
rocketmail.com         664
mail.com               559
earthlink.net          514
juno.com               322
aim.com                315
windstream.net         305
roadrunner.com         305
frontier.com           280
embarqmail.com         260
web.de                 240
netzero.com            230
twc.com                230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
cfl.rr.com             172
sc.rr.com              164
cableone.net           159
g

In [288]:
df_train['P_emaildomain'][df_train['P_emaildomain'].str.contains('me') == True] = 'icloud'
df_train.P_emaildomain.value_counts()

gmail               228851
yahoo               102909
outlook              59477
anonymous            36998
aol                  28289
icloud                8225
comcast.net           7888
att.net               4033
sbcglobal.net         2970
verizon.net           2705
ymail.com             2396
bellsouth.net         1909
cox.net               1393
optonline.net         1011
charter.net            816
rocketmail.com         664
mail.com               559
earthlink.net          514
juno.com               322
aim.com                315
roadrunner.com         305
windstream.net         305
frontier.com           280
embarqmail.com         260
web.de                 240
twc.com                230
netzero.com            230
prodigy.net.mx         207
centurylink.net        205
netzero.net            196
frontiernet.net        195
q.com                  189
suddenlink.net         175
cfl.rr.com             172
sc.rr.com              164
cableone.net           159
gmx.de                 149
p

In [289]:
print('gmail: ', 228851/590540*100, '%')
print('yahoo: ', 102909/590540*100, '%')
print('outlook: ', 59477/590540*100, '%')
print('anonymous: ', 36998/590540*100, '%')
print('aol: ', 28289/590540*100, '%')
print('icloud: ', 8225/590540*100, '%')
print()
print(round((228851+102909+59477+36998+28289+8225)/590540 *100, 2), '%')


gmail:  38.75283638703559 %
yahoo:  17.42625393707454 %
outlook:  10.071629356182477 %
anonymous:  6.265113286144884 %
aol:  4.790361364175162 %
icloud:  1.3927930368814985 %

78.7 %


In [290]:
df_train.loc[~df_train['P_emaildomain'].isin([
                                    'gmail',
                                    'yahoo',
                                    'outlook',
                                    'anonymous',
                                    'aol',
                                    'icloud',
                                    ]),'P_emaildomain'] = 'other'

In [291]:
df_train.P_emaildomain.value_counts()
#df_train.P_emaildomain.unique()
#len(df_train.P_emaildomain)  # 590540

gmail        228851
other        125791
yahoo        102909
outlook       59477
anonymous     36998
aol           28289
icloud         8225
Name: P_emaildomain, dtype: int64

#### **R_emaildomain (60) 변수 줄이기 → 7개로 줄이기**
- gmail
- yahoo
- outlook
- anonymous
- aol
- icloud
- other

In [292]:
df_train.R_emaildomain.value_counts()

gmail.com           57147
hotmail.com         27509
anonymous.com       20529
yahoo.com           11842
aol.com              3701
outlook.com          2507
comcast.net          1812
yahoo.com.mx         1508
icloud.com           1398
msn.com               852
live.com              762
live.com.mx           754
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
outlook.es            433
att.net               430
bellsouth.net         422
hotmail.fr            293
hotmail.es            292
web.de                237
mac.com               218
ymail.com             207
prodigy.net.mx        207
optonline.net         187
gmx.de                147
yahoo.fr              137
charter.net           127
mail.com              122
hotmail.co.uk         105
gmail                  95
earthlink.net          79
yahoo.de               75
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
yahoo.es               57
live.fr     

In [293]:
df_train.R_emaildomain.loc[df_train.R_emaildomain.str.contains('gmail') == True] = 'gmail'
df_train.R_emaildomain.value_counts()

gmail               57242
hotmail.com         27509
anonymous.com       20529
yahoo.com           11842
aol.com              3701
outlook.com          2507
comcast.net          1812
yahoo.com.mx         1508
icloud.com           1398
msn.com               852
live.com              762
live.com.mx           754
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
outlook.es            433
att.net               430
bellsouth.net         422
hotmail.fr            293
hotmail.es            292
web.de                237
mac.com               218
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
yahoo.fr              137
charter.net           127
mail.com              122
hotmail.co.uk         105
earthlink.net          79
yahoo.de               75
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
yahoo.es               57
live.fr                55
roadrunner.c

In [294]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('yahoo') == True] = 'yahoo'
df_train.R_emaildomain.value_counts()

gmail               57242
hotmail.com         27509
anonymous.com       20529
yahoo               13691
aol.com              3701
outlook.com          2507
comcast.net          1812
icloud.com           1398
msn.com               852
live.com              762
live.com.mx           754
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
outlook.es            433
att.net               430
bellsouth.net         422
hotmail.fr            293
hotmail.es            292
web.de                237
mac.com               218
ymail.com             207
prodigy.net.mx        207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
hotmail.co.uk         105
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
live.fr                55
roadrunner.com         53
juno.com               53
frontier.com           52
windstream.net         47
hotmail.de  

In [295]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('outlook') == True] = 'outlook'
df_train.R_emaildomain.value_counts()

gmail               57242
hotmail.com         27509
anonymous.com       20529
yahoo               13691
aol.com              3701
outlook              2940
comcast.net          1812
icloud.com           1398
msn.com               852
live.com              762
live.com.mx           754
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
hotmail.fr            293
hotmail.es            292
web.de                237
mac.com               218
ymail.com             207
prodigy.net.mx        207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
hotmail.co.uk         105
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
live.fr                55
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
hotmail.de             42
protonmail.c

In [296]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('hotmail') == True] = 'outlook'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             31181
anonymous.com       20529
yahoo               13691
aol.com              3701
comcast.net          1812
icloud.com           1398
msn.com               852
live.com              762
live.com.mx           754
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
mac.com               218
ymail.com             207
prodigy.net.mx        207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
live.fr                55
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
ptd.net     

In [297]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('msn') == True] = 'outlook'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             32033
anonymous.com       20529
yahoo               13691
aol.com              3701
comcast.net          1812
icloud.com           1398
live.com              762
live.com.mx           754
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
mac.com               218
ymail.com             207
prodigy.net.mx        207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
live.fr                55
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net     

In [298]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('live') == True] = 'outlook'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             33604
anonymous.com       20529
yahoo               13691
aol.com              3701
comcast.net          1812
icloud.com           1398
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
mac.com               218
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net                27
suddenlink.net         25
q.com                  25
netzero.com 

In [299]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('anonymous') == True] = 'anonymous'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             33604
anonymous           20529
yahoo               13691
aol.com              3701
comcast.net          1812
icloud.com           1398
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
mac.com               218
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net                27
suddenlink.net         25
q.com                  25
netzero.com 

In [300]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('aol') == True] = 'aol'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             33604
anonymous           20529
yahoo               13691
aol                  3701
comcast.net          1812
icloud.com           1398
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
mac.com               218
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net                27
suddenlink.net         25
q.com                  25
netzero.com 

In [301]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('icloud') == True] = 'icloud'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             33604
anonymous           20529
yahoo               13691
aol                  3701
comcast.net          1812
icloud               1398
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
mac.com               218
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net                27
suddenlink.net         25
q.com                  25
netzero.com 

In [302]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('mac') == True] = 'icloud'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             33604
anonymous           20529
yahoo               13691
aol                  3701
comcast.net          1812
icloud               1616
verizon.net           620
me.com                556
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net                27
suddenlink.net         25
q.com                  25
netzero.com            14
frontiernet.

In [303]:
df_train['R_emaildomain'][df_train['R_emaildomain'].str.contains('me.') == True] = 'icloud'
df_train.R_emaildomain.value_counts()

gmail               57242
outlook             33604
anonymous           20529
yahoo               13691
aol                  3701
icloud               2172
comcast.net          1812
verizon.net           620
sbcglobal.net         552
cox.net               459
att.net               430
bellsouth.net         422
web.de                237
prodigy.net.mx        207
ymail.com             207
optonline.net         187
gmx.de                147
charter.net           127
mail.com              122
earthlink.net          79
rocketmail.com         69
embarqmail.com         68
scranton.edu           63
juno.com               53
roadrunner.com         53
frontier.com           52
windstream.net         47
protonmail.com         41
cfl.rr.com             37
aim.com                36
servicios-ta.com       35
twc.com                29
cableone.net           27
ptd.net                27
suddenlink.net         25
q.com                  25
netzero.com            14
frontiernet.net        14
centurylink.

In [304]:
print('gmail: ', 57242/590540*100, '%')
print('outlook: ', 33604/590540*100, '%')
print('anonymous: ', 20529/590540*100, '%')
print('yahoo: ', 13691/590540*100, '%')
print('aol: ', 3701/590540*100, '%')
print('icloud: ', 2172/590540*100, '%')
print()
print(round((57242+33604+20529+13691+3701+2172)/590540 *100, 2), '%')

     

gmail:  9.693162190537475 %
outlook:  5.690385071290683 %
anonymous:  3.4763098181325565 %
yahoo:  2.3183865614522303 %
aol:  0.6267145324618145 %
icloud:  0.3677989636603786 %

22.17 %


In [305]:
df_train.loc[~df_train['R_emaildomain'].isin([
                                    'gmail',
                                    'yahoo',
                                    'anonymous',
                                    'aol',
                                    'outlook',
                                    'icloud',
                                    ]),'R_emaildomain'] = 'other'

In [306]:
df_train.R_emaildomain.value_counts()#
#df_train.R_emaildomain.unique()
#len(df_train.R_emaildomain)  # 

other        459601
gmail         57242
outlook       33604
anonymous     20529
yahoo         13691
aol            3701
icloud         2172
Name: R_emaildomain, dtype: int64

## X, Target column 구분

In [307]:
tsa_x = train_transaction.drop(columns=["isFraud"])
tsa_y = train_transaction["isFraud"]

In [308]:
tsa_x

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,W,6550,NaN,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [309]:
tsa_y

0         0
1         0
2         0
3         0
4         0
         ..
590535    0
590536    0
590537    0
590538    0
590539    0
Name: isFraud, Length: 590540, dtype: int64